In [1]:
%matplotlib inline
import platform
print('the python version is: ' + platform.python_version())
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext

the python version is: 3.6.3


In [2]:
print('\n' + ' //// Started local Spark session //// ' + '\n')
sc = SparkContext("local", "RDD basic functions App")
sc.setLogLevel("WARN")


 //// Started local Spark session //// 



Load data from World Wide Web and store it in *kddcup.data_10_percent.gz*:

Information about data schema can be found here: http://kdd.ics.uci.edu/databases/kddcup99/task.html

In [3]:
import urllib.request
f = urllib.request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")
data_file = "./kddcup.data_10_percent.gz"
#construct the RDD from input file
rawData = sc.textFile(data_file)

Show first 5 lines:

In [4]:
firstFiveLines = rawData.take(5)
for line in firstFiveLines:
    print(line)

0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.
0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.


In [5]:
#filter out normal connections - transformation
normalData = rawData.filter(lambda line: 'normal.' in line)

In [6]:
#filter out normal connections - transformation + action
normalDataCount = normalData.count()
print(normalDataCount)

97278


**Exercise:** Count the "normal" entries in connection data set and compare to total number 

In [7]:
rawDataCount = rawData.count()
print('the total number of entries in data set is: {}'.format(rawDataCount) + ' compared to {} '.format(normalDataCount) + 'normal entries.')

the total number of entries in data set is: 494021 compared to 97278 normal entries.


**Exercise:** Measure the time needed to formulate transformation and combination to transformation (filter) plus action (counting). Hint: use the 'time' module

In [8]:
#measure time to complete full map reduce
from time import time
t0 = time()
normalData = rawData.filter(lambda x: 'normal.' in x)
t1 = time()
t_trans = t1-t0
t0 = time()
normalDataCount = rawData.filter(lambda x: 'normal.' in x).collect()
t1 = time()
t_transAct = t1-t0
print('transformation time: {}'.format(round((t_trans)*1000,3)) + ' ms, full time: {}'.format(round((t_transAct)*1000,3)) +' ms')

transformation time: 0.055 ms, full time: 745.001 ms


To avoid overhead, sometimes it is sufficient to get an approximation for an amount. For this, a sub-sample or the data set might be sufficient to get approximate values. 

**Exercise:** Use the 'sample' transformation to collect a subset of the full data set and repeat time measurement. What is your observation? Vary the sample size starting with 10%.

In [9]:
t0 = time()
rawDataSamples = rawData.sample(False, 0.1, 1234)
samplesCount = rawDataSamples.count()
t1 = time()
t_sample = t1-t0
print('Sample size is {}'.format(samplesCount) + ' of {}'.format(rawDataCount))
print('sample full time: {}'.format(round((t_sample)*1000,3)) + ' ms, full time: {}'.format(round((t_transAct)*1000,3)) +' ms')

Sample size is 49493 of 494021
sample full time: 536.54 ms, full time: 745.001 ms


Get the suspicious data sets by using the 'subtract' transformation.

In [10]:
suspiciousData = rawData.subtract(normalData)
suspiciousDataCount = suspiciousData.count()
print('Number of suspicious entries  {}'.format(suspiciousDataCount))

Number of suspicious entries  396743


**Exercise:** Collect all different types of connection protocols and used services in the data set (you can also use the sample data set instead of the full one). Hint: you have to first split up each individual entry, since it is simply a long string. Use the 'split()' function in a mapping transformation first.

In [11]:
inData = rawData.map(lambda line: line.split(","))
print('take first element each')
print(rawData.take(1))
print(inData.take(1))

take first element each
['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.']
[['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]


In [12]:
protocols = inData.map(lambda x: x[1]).distinct()
print('all protcols found in the data set: {} '.format(protocols.collect()) )

all protcols found in the data set: ['tcp', 'udp', 'icmp'] 


In [13]:
services = inData.map(lambda x: x[2]).distinct()
print( services.collect() )

['http', 'smtp', 'finger', 'domain_u', 'auth', 'telnet', 'ftp', 'eco_i', 'ntp_u', 'ecr_i', 'other', 'private', 'pop_3', 'ftp_data', 'rje', 'time', 'mtp', 'link', 'remote_job', 'gopher', 'ssh', 'name', 'whois', 'domain', 'login', 'imap4', 'daytime', 'ctf', 'nntp', 'shell', 'IRC', 'nnsp', 'http_443', 'exec', 'printer', 'efs', 'courier', 'uucp', 'klogin', 'kshell', 'echo', 'discard', 'systat', 'supdup', 'iso_tsap', 'hostnames', 'csnet_ns', 'pop_2', 'sunrpc', 'uucp_path', 'netbios_ns', 'netbios_ssn', 'netbios_dgm', 'sql_net', 'vmnet', 'bgp', 'Z39_50', 'ldap', 'netstat', 'urh_i', 'X11', 'urp_i', 'pm_dump', 'tftp_u', 'tim_i', 'red_i']


In [14]:
product = protocols.cartesian(services).collect()
print('There are {} combinations of protocol X service'.format(len(product)))

There are 198 combinations of protocol X service


In [15]:
combinations = inData.map(lambda x: (x[1],x[2])).distinct()
print('There are {} combinations of protocols and services'.format(combinations.count()))

There are 68 combinations of protocols and services


Show all protocol/service combinations which do not exist in the data set:

In [16]:
print(protocols.cartesian(services).subtract(combinations).collect())

[('tcp', 'eco_i'), ('tcp', 'urh_i'), ('tcp', 'urp_i'), ('tcp', 'red_i'), ('udp', 'http'), ('icmp', 'http'), ('udp', 'smtp'), ('icmp', 'smtp'), ('udp', 'auth'), ('icmp', 'domain_u'), ('icmp', 'auth'), ('udp', 'ecr_i'), ('udp', 'pop_3'), ('udp', 'ftp_data'), ('icmp', 'ntp_u'), ('icmp', 'pop_3'), ('icmp', 'ftp_data'), ('udp', 'time'), ('udp', 'mtp'), ('udp', 'link'), ('udp', 'gopher'), ('udp', 'ssh'), ('udp', 'whois'), ('udp', 'imap4'), ('udp', 'ctf'), ('udp', 'IRC'), ('icmp', 'time'), ('icmp', 'mtp'), ('icmp', 'link'), ('icmp', 'gopher'), ('icmp', 'ssh'), ('icmp', 'whois'), ('icmp', 'imap4'), ('icmp', 'ctf'), ('icmp', 'IRC'), ('udp', 'http_443'), ('udp', 'printer'), ('udp', 'courier'), ('udp', 'kshell'), ('udp', 'discard'), ('udp', 'systat'), ('udp', 'hostnames'), ('udp', 'csnet_ns'), ('udp', 'netbios_ns'), ('udp', 'netbios_ssn'), ('udp', 'netbios_dgm'), ('udp', 'bgp'), ('udp', 'Z39_50'), ('udp', 'pm_dump'), ('icmp', 'http_443'), ('icmp', 'printer'), ('icmp', 'courier'), ('icmp', 'kshell

In [17]:
sc.stop()